# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [128]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel) * np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        return np.power(kernel, 3)
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [129]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from sklearn.metrics import accuracy_score

iris = load_iris()
data_set = iris.data
labels = iris.target

In [130]:
def choose_set_for_label(data_set, labels, label):
    binary_labels = np.where(labels == label, 1, -1)
    return train_test_split(data_set, binary_labels, test_size=0.2, random_state=15)




In [131]:
def get_labels_count(labels):
    return len(np.unique(labels))

Use the code that we have implemented earlier:

In [132]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    global objects_count
    objects_count = len(train_data_set)

    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A),
                            cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(
            lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number


def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel) * np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel


def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors,
                 support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T)) * np.ones((1, len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv, np.ones((1, np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    # return np.sign(y)
    return y

In [133]:
from sklearn.metrics import accuracy_score

def prepare_binary_dataset(X, y, target_class):
    return choose_set_for_label(X, y, target_class)

def train_and_predict(train_X, train_y, test_X):
    model = train(train_X, train_y, kernel_type='rbf')
    predictions = classify_rbf(
        test_X, train_X,
        model[0], model[4], model[3], model[5],
        model[1], model[2]
    )
    return list(predictions.astype(int))

def evaluate_accuracy(true_labels, predicted_labels):
    return accuracy_score(true_labels, predicted_labels)

for class_id in range(get_labels_count(labels)):
    X_train, X_test, y_train, y_test = prepare_binary_dataset(data_set, labels, class_id)
    y_pred = train_and_predict(X_train, y_train, X_test)
    acc = evaluate_accuracy(y_test, y_pred)
    print(f"Label: {class_id}, accuracy: {acc}")

     pcost       dcost       gap    pres   dres
 0:  1.1726e+02 -1.7992e+03  4e+03  2e-01  2e-15
 1:  7.7967e+01 -1.6973e+02  3e+02  5e-03  3e-15
 2:  1.0169e+01 -2.2323e+01  3e+01  1e-15  3e-15
 3: -4.9607e-01 -4.9938e+00  4e+00  2e-16  1e-15
 4: -1.4344e+00 -2.5218e+00  1e+00  3e-16  4e-16
 5: -1.6897e+00 -2.1674e+00  5e-01  2e-16  3e-16
 6: -1.8078e+00 -2.0073e+00  2e-01  5e-16  3e-16
 7: -1.8464e+00 -1.9763e+00  1e-01  2e-16  4e-16
 8: -1.8825e+00 -1.9020e+00  2e-02  2e-16  4e-16
 9: -1.8902e+00 -1.8906e+00  4e-04  2e-16  4e-16
10: -1.8904e+00 -1.8904e+00  5e-06  4e-16  4e-16
11: -1.8904e+00 -1.8904e+00  5e-08  2e-16  4e-16
Optimal solution found.
Label: 0, accuracy: 0.0
     pcost       dcost       gap    pres   dres
 0:  1.1410e+02 -2.4247e+03  5e+03  2e-01  2e-15
 1:  9.0959e+01 -2.2448e+02  4e+02  6e-03  3e-15
 2:  1.3024e+01 -2.6284e+01  4e+01  4e-05  3e-15
 3: -1.1088e-01 -5.5940e+00  5e+00  4e-16  1e-15
 4: -1.3519e+00 -2.4130e+00  1e+00  2e-16  5e-16
 5: -1.5743e+00 -2.0232